# TEI Conference 2019 #
## Etape 2 : Normalisation des textes et préparation à l'analyse ##

Le but de cette deuxième partie du traitement est de normaliser les documents pour qu'ils soient tous semblables au moment de l'analyse. Cette chaîne de traitement va notamment produire des fichiers en texte brut à partir des abstracts en XML. J'ai choisis d'inclure les titres des ouvrages dans les abstracts, ce qui n'est pas le cas du document original. Cette décision est motivée par l'idée qu'un titre est une partie de choix du texte, et l'exclure lors d'une analyse textuelle serait une erreur.

### Les packages ###

Voici les packages dont nous aurons besoin. Il faut lancer la cellule ci-dessous une seule fois par lancement de ce notebook-ci.

In [1]:
import os
import csv
from lxml import etree
from tqdm import tqdm

### Création d'un dossier dans le cache ###

Avec la cellule ci-dessous, on initie un chemin relatif dans le dossier Python et on crée le dossier qui recevra les fichiers en texte brut.

In [2]:
Path = "./cache2019/cacheXML/"
Path_output = "./cache2019/cacheTXT/"
filelist = os.listdir(Path)
if not os.path.exists("./cache2019/cacheTXT"):
    os.makedirs("./cache2019/cacheTXT")

### Création des fichiers .txt ###

Les fichiers en XML enregistrés auparavant sont lus un par un dans la boucle for, le titre est stocké dans une variable et remis dans sa forme originale (les ' _ ' sont remplacés par des espaces comme c'était le cas initialement) pour être réutilisé plus loin.
Ensuite, le body du document XML est nettoyé de toutes ses balises et ses espaces (tabulations, retour à la ligne, etc...) en trop sont retirés.
Le titre est ajouté dans le body, car dans une étude statistique des mots employés, il m'a semblé que le titre est un espace de choix souvent très éloquent pour connaître le thème d'un texte, et il a donc de plein droit sa place dans les éléments analysés.

In [3]:
number = 1 # J'initialise un compteur qui servira d'identifiants à l'abstract.
dict_noms_abstract = {} # J'instancie un dictionnaire où la clé est l'identifiant complet de l'abstract et la valeur est le titre de l'abstract. Cela permettra d'ajouter une colonne au csv de correspondance avec le code des abstracts.
for abstract in tqdm(filelist):
    if abstract.endswith(".xml"): # C'est une sécurité, pour ne pas piocher n'importe quoi dans le cache, au cas où.
        titre = abstract # abstract est une str, et c'est le titre du fichier xml. Il va être nettoyé.
        titre = titre.replace(".xml", "")
        titre = titre.replace("_", " ")
        try: #permet d'éviter les erreurs qui bloqueraient tout.
            with open(Path + abstract, "r", encoding="UTF-8") as y: #Ici, on nettoie le texte de ses balises, car elles n'ont aucun intérêt dans un usage d'analyse statistique sémantique.
                tree = etree.parse(y) 
                notags = etree.tostring(tree, encoding='utf8', method='text') #notags est une str géante en ASCII.
                year = 2019
                abstract = "paper-{0}-{1}.txt".format(year, number) #Ceci est l'identifiant unique du document texte .txt
                input_name = abstract # input_name est également une str, mais que l'on va directement ajouter dans le document.
                input_name = input_name.replace(".txt", "")
                notags = notags.decode('UTF-8') 
                liste_mots = notags.split() # l'enchaînement .split / .join permet de retirer les espaces inutiles laissés par la suppression des balises
                texte = ' '.join(liste_mots)
                dict_noms_abstract[titre] = input_name
                with open(Path_output + abstract, "w", encoding="UTF-8") as z:
                    titre = titre + " "
                    z.write(str(titre))
                    z.write(str(texte))
                number += 1
        except Exception:
            raise
            pass

100%|██████████| 46/46 [00:00<00:00, 190.56it/s]


### Ajout d'une grille de correspondance ###

Afin de pouvoir retrouver l'abstract à partir des métadonnées stockées dans le fichier csv, il faut ajouter une colonne constituée des identifiants des abstracts.
Pour cela, un dictionnaire a été construit ( dict_noms_abstract ) où la clé est le titre réel de l'abstract (celui donné par l'auteur et stocké dans le csv) et où la valeur est l'identifiant que j'ai attribué à chaque abstract dans le cache.
Ne pouvant ajouter dans le document d'origine cette colonne, je suis obligé de copier le document, puis supprimer l'ancien et renommer le nouveau avec le nom de l'ancien.

In [8]:
with open('./cache2019/TEI2019.csv', 'r') as csvinput:
    with open('./cache2019/TEI2019-2.csv', 'w') as csvoutput:
        writer = csv.writer(csvoutput, lineterminator='\n')
        reader = csv.reader(csvinput)
        ensemble_du_document = [] # J'instancie une liste qui recevra tous les éléments du csv d'origine + la valeur correspondante du dictionnaire
        for row in reader:
            if row[2] in dict_noms_abstract:
                row.append(dict_noms_abstract[row[2]])
                ensemble_du_document.append(row)
        writer.writerows(ensemble_du_document)

if os.path.exists('./cache2019/TEI2019.csv'):
    os.remove('./cache2019/TEI2019.csv')
else:
    print("Fichier inexistant")

if os.path.exists('./cache2019/TEI2019-2.csv'):
    os.rename('./cache2019/TEI2019-2.csv', './cache2019/TEI2019.csv')
else:
    print("Fichier inexistant")